In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI

template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)


chatgpt_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, streaming = True),
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=2)
)

def chat(message):
    output = chatgpt_chain.predict(human_input = message)
    
    return output

In [ ]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
response = chat ("hello")
printmd(response)

In [ ]:
import gradio as gr

class ChatBot:
    def __init__(self):
        self.history = []

    def bot(self, user_message, history):
        if isinstance(user_message, list) and len(user_message) > 0:
            last_message = user_message[-1][0]
        else:
            last_message = str(user_message)

        bot_message = chat(last_message)
        self.history.append([last_message, bot_message])
        return self.history

chatbot_instance = ChatBot()

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def user(user_message, history):
        return gr.update(value="", interactive=False), history + [[user_message, None]]

    send_message = gr.Button("Send Message")
    send_message.click(chatbot_instance.bot, msg, chatbot)
    response = msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_instance.bot, chatbot, chatbot
    )
    response.then(lambda: gr.update(interactive=True), None, [msg], queue=False)

demo.queue()
demo.launch()
